Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

## Logistic

In [77]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + 0.002*tf.nn.l2_loss(weights) + 0.0002*tf.nn.l2_loss(biases)) 

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [78]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  session.run(tf.initialize_all_variables())
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 22.810183
Minibatch accuracy: 16.4%
Validation accuracy: 14.2%
Minibatch loss at step 500: 2.991053
Minibatch accuracy: 80.5%
Validation accuracy: 76.7%
Minibatch loss at step 1000: 1.452766
Minibatch accuracy: 77.3%
Validation accuracy: 79.7%
Minibatch loss at step 1500: 0.902496
Minibatch accuracy: 79.7%
Validation accuracy: 82.0%
Minibatch loss at step 2000: 0.905476
Minibatch accuracy: 79.7%
Validation accuracy: 81.4%
Minibatch loss at step 2500: 0.633796
Minibatch accuracy: 82.0%
Validation accuracy: 82.0%
Minibatch loss at step 3000: 0.662966
Minibatch accuracy: 83.6%
Validation accuracy: 82.0%
Test accuracy: 88.7%


## Neural Network

In [79]:
batch_size = 128
n_hidden = 1024

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

def multi_layer_perceptron(data, weights, biases):
  # Training computation.
  hidden1 = tf.matmul(data, weights['h1']) + biases['h1']
  hidden1 = tf.nn.relu(hidden1)
  output = tf.matmul(hidden1, weights['out']) + biases['out']
  
  return output

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = {
    'h1': tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden])),
    'out': tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  }
  biases = {
    'h1': tf.Variable(tf.zeros([n_hidden])),
    'out': tf.Variable(tf.zeros([num_labels]))
  }

  out = multi_layer_perceptron(tf_train_dataset, weights, biases)
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(out, tf_train_labels) 
    + 0.002*tf.nn.l2_loss(weights['h1']) + 0.0002*tf.nn.l2_loss(biases['h1']) 
    + 0.002*tf.nn.l2_loss(weights['out']) + 0.0002*tf.nn.l2_loss(biases['out']))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(out)
  valid_prediction = tf.nn.softmax(multi_layer_perceptron(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(multi_layer_perceptron(tf_test_dataset, weights, biases))

In [80]:
num_steps = 3001

test_predictions = []

with tf.Session(graph=graph) as session:
  session.run(tf.initialize_all_variables())
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    
  test_predictions = test_prediction.eval()
  print("Test accuracy: %.1f%%" % accuracy(test_predictions, test_labels))

Initialized
Minibatch loss at step 0: 970.947876
Minibatch accuracy: 3.9%
Validation accuracy: 28.9%
Minibatch loss at step 500: 231.107666
Minibatch accuracy: 82.0%
Validation accuracy: 78.8%
Minibatch loss at step 1000: 84.450119
Minibatch accuracy: 82.8%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 30.982683
Minibatch accuracy: 85.9%
Validation accuracy: 86.2%
Minibatch loss at step 2000: 11.862658
Minibatch accuracy: 85.2%
Validation accuracy: 87.1%
Minibatch loss at step 2500: 4.714274
Minibatch accuracy: 88.3%
Validation accuracy: 87.2%
Minibatch loss at step 3000: 2.135167
Minibatch accuracy: 86.7%
Validation accuracy: 87.3%
Test accuracy: 92.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [106]:
batch_size = 528
n_hidden = 1024

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

def multi_layer_perceptron(data, weights, biases):
  # Training computation.
  hidden1 = tf.matmul(data, weights['h1']) + biases['h1']
  hidden1 = tf.nn.relu(hidden1)
  output = tf.matmul(hidden1, weights['out']) + biases['out']
  
  return output

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = {
    'h1': tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden])),
    'out': tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  }
  biases = {
    'h1': tf.Variable(tf.zeros([n_hidden])),
    'out': tf.Variable(tf.zeros([num_labels]))
  }

  out = multi_layer_perceptron(tf_train_dataset, weights, biases)
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(out, tf_train_labels) 
    + 0.002*tf.nn.l2_loss(weights['h1']) + 0.0002*tf.nn.l2_loss(biases['h1']) 
    + 0.002*tf.nn.l2_loss(weights['out']) + 0.0002*tf.nn.l2_loss(biases['out']))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(out)
  valid_prediction = tf.nn.softmax(multi_layer_perceptron(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(multi_layer_perceptron(tf_test_dataset, weights, biases))

In [107]:
num_steps = 3

test_predictions = []

with tf.Session(graph=graph) as session:
  session.run(tf.initialize_all_variables())
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    
  test_predictions = test_prediction.eval()
  print("Test accuracy: %.1f%%" % accuracy(test_predictions, test_labels))

Initialized
Minibatch loss at step 0: 977.309998
Minibatch accuracy: 9.3%
Validation accuracy: 32.1%
Minibatch loss at step 1: 1621.397217
Minibatch accuracy: 34.1%
Validation accuracy: 41.8%
Minibatch loss at step 2: 2417.629395
Minibatch accuracy: 40.0%
Validation accuracy: 38.1%
Test accuracy: 41.3%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [178]:
batch_size = 128
n_hidden = 1024

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

def multi_layer_perceptron(data, weights, biases, keep_prob):
  # Training computation.
  hidden1 = tf.matmul(data, weights['h1']) + biases['h1']
  hidden1 = tf.nn.relu(hidden1)
  hidden1 = tf.nn.dropout(hidden1,keep_prob)
  #if val is True:
  #  hidden1 = hidden1 * (1.0/keep_prob)
    
  output = tf.matmul(hidden1, weights['out']) + biases['out']

  return output

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = {
    'h1': tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden])),
    'out': tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  }
  biases = {
    'h1': tf.Variable(tf.zeros([n_hidden])),
    'out': tf.Variable(tf.zeros([num_labels]))
  }
  keep_prob = tf.placeholder(tf.float32)

  out = multi_layer_perceptron(tf_train_dataset, weights, biases, keep_prob)
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(out, tf_train_labels) 
    + 0.002*tf.nn.l2_loss(weights['h1']) + 0.0002*tf.nn.l2_loss(biases['h1']) 
    + 0.002*tf.nn.l2_loss(weights['out']) + 0.0002*tf.nn.l2_loss(biases['out']))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(out)
  valid_prediction = tf.nn.softmax(multi_layer_perceptron(tf_valid_dataset, weights, biases, keep_prob))
  test_prediction = tf.nn.softmax(multi_layer_perceptron(tf_test_dataset, weights, biases, keep_prob))

In [179]:
num_steps = 3001

test_predictions = []

with tf.Session(graph=graph) as session:
  session.run(tf.initialize_all_variables())
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict = {keep_prob : 1.0}), valid_labels))
    
  test_predictions = test_prediction.eval(feed_dict = {keep_prob : 1.0})
  print("Test accuracy: %.1f%%" % accuracy(test_predictions, test_labels))

Initialized
Minibatch loss at step 0: 1048.203979
Minibatch accuracy: 12.5%
Validation accuracy: 33.2%
Minibatch loss at step 500: 235.859131
Minibatch accuracy: 77.3%
Validation accuracy: 79.8%
Minibatch loss at step 1000: 85.725067
Minibatch accuracy: 75.8%
Validation accuracy: 79.9%
Minibatch loss at step 1500: 31.390869
Minibatch accuracy: 76.6%
Validation accuracy: 84.8%
Minibatch loss at step 2000: 11.936215
Minibatch accuracy: 82.8%
Validation accuracy: 86.0%
Minibatch loss at step 2500: 4.925150
Minibatch accuracy: 81.2%
Validation accuracy: 86.3%
Minibatch loss at step 3000: 2.262563
Minibatch accuracy: 85.2%
Validation accuracy: 86.6%
Test accuracy: 92.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [201]:
batch_size = 128
n_hidden = 1024

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

def multi_layer_perceptron(data, weights, biases, keep_prob):
  # Training computation.
  hidden1 = tf.matmul(data, weights['h1']) + biases['h1']
  hidden1 = tf.nn.relu(hidden1)
  #hidden1 = tf.nn.dropout(hidden1,keep_prob)
  #if val is True:
  #  hidden1 = hidden1 * (1.0/keep_prob)
    
  output = tf.matmul(hidden1, weights['out']) + biases['out']

  return output

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = {
    'h1': tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden])),
    'out': tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  }
  biases = {
    'h1': tf.Variable(tf.zeros([n_hidden])),
    'out': tf.Variable(tf.zeros([num_labels]))
  }
  keep_prob = tf.placeholder(tf.float32)

  out = multi_layer_perceptron(tf_train_dataset, weights, biases, keep_prob)
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(out, tf_train_labels) 
    + 0.002*tf.nn.l2_loss(weights['h1']) + 0.0002*tf.nn.l2_loss(biases['h1']) 
    + 0.002*tf.nn.l2_loss(weights['out']) + 0.0002*tf.nn.l2_loss(biases['out']))
  
  # Optimizer.
  #global_step = tf.Variable(0)  # count the number of steps taken.
  #learning_rate = tf.train.exponential_decay(0.5, global_step, 3001, 0.96, staircase=True)
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(out)
  valid_prediction = tf.nn.softmax(multi_layer_perceptron(tf_valid_dataset, weights, biases, keep_prob))
  test_prediction = tf.nn.softmax(multi_layer_perceptron(tf_test_dataset, weights, biases, keep_prob))

In [202]:
num_steps = 3001

test_predictions = []

with tf.Session(graph=graph) as session:
  session.run(tf.initialize_all_variables())
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict = {keep_prob : 1.0}), valid_labels))
    
  test_predictions = test_prediction.eval(feed_dict = {keep_prob : 1.0})
  print("Test accuracy: %.1f%%" % accuracy(test_predictions, test_labels))

Initialized
Minibatch loss at step 0: 950.548889
Minibatch accuracy: 12.5%
Validation accuracy: 31.9%
Minibatch loss at step 500: 232.944199
Minibatch accuracy: 79.7%
Validation accuracy: 80.3%
Minibatch loss at step 1000: 84.394455
Minibatch accuracy: 79.7%
Validation accuracy: 80.9%
Minibatch loss at step 1500: 30.898659
Minibatch accuracy: 86.7%
Validation accuracy: 86.0%
Minibatch loss at step 2000: 11.859541
Minibatch accuracy: 85.9%
Validation accuracy: 87.0%
Minibatch loss at step 2500: 4.697701
Minibatch accuracy: 87.5%
Validation accuracy: 87.0%
Minibatch loss at step 3000: 2.118799
Minibatch accuracy: 85.9%
Validation accuracy: 87.8%
Test accuracy: 93.0%
